# Welcome this will be where all our info goes

In [236]:
# Importing all the goodies
import pandas as pd
import numpy as np


# ploting imports
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import openpyxl as pxl


# All Data to import using dataframes
population_Data_Path = "../data/population.csv"
Electricity_Generation_Data_Path = "../data/Electricity_Generation.xlsx"
Electricity_Interchange_Data_Path = "../data/Electricity_Interchange.xlsx"
Electricity_Capacity_Data_Path = "../data/Electricity_Capacity.xlsx"
End_Use_Demand_Data_Path = "../data/End_Use_Demand.xlsx"
End_Use_Prices_Data_Path = "../data/End_Use_Prices.xlsx"
energy_trade_canada_path = "../data/energy_trade_canada.xls"
Canadian_Climate_Normals_CANADA_Data_Path = "../data/1991-2020_Canadian_Climate_Normals_CANADA_Data.csv"

## Load data sets


In [237]:



# CSV files
population_df = pd.read_csv(population_Data_Path, sep=",", skiprows=3, header=0, engine="python", quotechar='"', on_bad_lines="skip")
climate_df = pd.read_csv(Canadian_Climate_Normals_CANADA_Data_Path, sep=",")

# # Excel Files
generation_df = pd.read_excel(Electricity_Generation_Data_Path)
capacity_df = pd.read_excel(Electricity_Capacity_Data_Path)
interchange_df = pd.read_excel(Electricity_Interchange_Data_Path)
demand_df = pd.read_excel(End_Use_Demand_Data_Path)
prices_df = pd.read_excel(End_Use_Prices_Data_Path)
# prtrade_df = pd.read_excel(energy_trade_canada_path)


dataframes = {
    "Population": population_df,
    "Generation": generation_df,
    "Capacity": capacity_df,
    "Interchange": interchange_df,
    "Demand": demand_df,
    "Prices": prices_df,
    "Climate": climate_df
}





### Power Generation Data Set Cleaning 

In [238]:

def clean_generation(df, start_row, province_name, num_rows = 8):

    header_row = start_row + 1
    data_rows = start_row + 2

    headers = df.iloc[header_row]
    block = df.iloc[data_rows : data_rows + num_rows].copy()
    block.columns = headers

    block = block.dropna(subset=[block.columns[0]])

    block = block.rename(columns={block.columns[0]: "Energy_Type"})
    block = block.melt(id_vars="Energy_Type", var_name="Year", value_name="GWh")

    block["Province"] = province_name
    block["Year"] = pd.to_numeric(block["Year"], errors="coerce")
    block["GWh"] = pd.to_numeric(block["GWh"], errors="coerce")

    block = block.dropna(subset=["Year", "GWh"])
    block["Year"] = block["Year"].astype(int)

    return block

provinces_generation = {
    "Canada": 5,
    "Newfoundland and Labrador": 16,
    "Prince Edward Island": 27,
    "Nova Scotia": 38,
    "New Brunswick": 49,
    "Quebec": 60,
    "Ontario": 71,
    "Manitoba": 82,
    "Alberta": 93,
    "British Columbia": 104,
    "Saskatchewan": 115,
    "Yukon": 126,
    "Northwest Territories": 137,
    "Nunavut": 148,
}

all_province_data = []

for province, start_row in provinces_generation.items():
    cleaned = clean_generation(generation_df, start_row, province)
    all_province_data.append(cleaned)


gen_clean = pd.concat(all_province_data, ignore_index=True)
quebec_df = gen_clean[gen_clean["Province"] == "Canada"]
quebec_df




,Energy_Type,Year,GWh,Province
0,Hydro / Wave / Tidal,2005,358380.80,Canada
1,Wind,2005,1453.41,Canada
2,Biomass / Geothermal,2005,6997.43,Canada
3,Solar,2005,0.00,Canada
4,Uranium,2005,86668.58,Canada
...,...,...,...,...
363,Solar,2050,62977.35,Canada
364,Uranium,2050,87524.19,Canada
365,Coal & Coke,2050,656.02,Canada
366,Natural Gas,2050,102350.23,Canada


### Power Capacity Data Set Cleaning

In [ ]:
# Im using the same function from generation data

provinces_cap = {
    "Canada": 5,
    "Newfoundland and Labrador": 16,
    "Prince Edward Island": 27,
    "Nova Scotia": 38,
    "New Brunswick": 49,
    "Quebec": 60,
    "Ontario": 71,
    "Manitoba": 82,
    "Alberta": 93,
    "British Columbia": 104,
    "Saskatchewan": 115,
    "Yukon": 126,
    "Northwest Territories": 137,
    "Nunavut": 148,
}

all_province_cap_data = []
for province, start_row in provinces_cap.items():
    cap_cleaned = clean_generation(capacity_df, start_row, province)
    all_province_cap_data.append(cap_cleaned)

cap_clean = pd.concat(all_province_cap_data, ignore_index=True)
# cap_clean

,Energy_Type,Year,GWh,Province
0,Hydro / Wave / Tidal,2005,72877.98,Canada
1,Wind,2005,557.37,Canada
2,Biomass / Geothermal,2005,1735.59,Canada
3,Solar,2005,16.75,Canada
4,Uranium,2005,12805.00,Canada
...,...,...,...,...
4496,Solar,2049,40.03,Nunavut
4497,Oil,2049,70.28,Nunavut
4498,Wind,2050,31.39,Nunavut
4499,Solar,2050,39.03,Nunavut


### Population Data Set Cleaning

In [240]:
print(population_df.columns[:5])
display(population_df.head())
def cleanPopulation(df):

    df = df.rename(columns={df.columns[0]: "Province"})
    df = df[~df["Province"].isna()]

    df_long = df.melt(id_vars="Province", var_name="Quarter", value_name="Population")
    df_long["Year"] = df_long["Quarter"].str.extract(r"(\d{4})")
    df_long = df_long.dropna(subset=["Year"])
    df_long["Year"] = df_long["Year"].astype(int)

    df_long["Population"] = df_long["Population"].replace(",", "", regex=True).astype(float)
    df_yearly = (
        df_long.groupby(["Province", "Year"], as_index=False)
        .agg({"Population": "mean"})
    )

    return df_yearly

population_yearly_df = cleanPopulation(population_df)

population_yearly_df

Index(['Release date: 2025-03-19'], dtype='object')


,Release date: 2025-03-19
0,"Geography: Canada, Province or territory"
1,Footnotes:
2,How to cite: Statistics Canada. Table 17-10-00...
3,https://www150.statcan.gc.ca/t1/tbl1/en/tv.act...


,Province,Year,Population
